## 24 July 2018
-- Laurin Gray

This notebook is to create a catalogue of all coordinate and magnitude info on the sources that have been selected as "red candidates" based on their position rightward of a 3-sigma line, in tiers of their likelyhood of being genuine AGBs.

The sources come from the catalog of Spitzer sources of Khan et al. (2015), matched with sources from Whitelock et al. (2013) in CasJobs.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import gaussian_kde
import csv
import pathlib

In [2]:
# Read in my data from a .csv file saved locally.

# all sources
phot_data = pd.read_csv('~/Documents/Phot_data/CMDparameters26June2018_lauringray.csv')

# CMD counts
CMD_counts = pd.read_csv('/Users/lgray/Documents/Phot_data/CMD_counting_23July2018_lauringray.csv')

In [3]:
# CMD counts
in_one = CMD_counts.in_one.values
in_two = CMD_counts.in_two.values
in_three = CMD_counts.in_three.values
in_four = CMD_counts.in_four.values
in_five = CMD_counts.in_five.values
in_six = CMD_counts.in_six.values
in_seven = CMD_counts.in_seven.values
in_eight = CMD_counts.in_eight.values

rows = phot_data.ID.values

In [4]:
def corr_rows(groups, lengths):
    """
    Some of the ID numbers are wrong (ex. there are two 2118s), which means we can't use the ID to 
    directly access the row it belongs to. As we go further down, the problem gets worse.
    This function finds the correct rows in phot_data for each ID and saves them to a list.

    Takes a list of the CMD counts you want to include in the plot (group) (and their corresponding lengths) 
    and outputs a list of the rows in phot_data which correspond to the IDs in those groups.
    
    Note that because groups must be a list, even if you are just running one column, 
    you need to define the column and length in a list first (ex. groups = [in_ten]; corr_rows(groups, lengths))
    
    """
    
    phot_rows = []
    
    #k = 0
    d = 0
    for j in groups:
        group_lim = lengths[d]
        k = 0
        #print(group_lim)
        for i in j:
            c = 0 # counter for phot_data rows, resets for each new element i
            # use a while loop so that it iterates until the end of the column
            while c < 30761 and k < group_lim: # to prevent reaching the end of the column and getting a nan error
                if int(i) != rows[c]: # check if i is equivalent to the ID in the current phot_data row
                    c = c+1 # if not, move to next row & go back to the top of the while loop
                else: # if i IS equivalent
                    phot_rows.append(c) # add the current row to corr_rows
                    c = 30761 # set c to stop iterating through the rest of the rows (end loop)
            k = k+1 # symbolically move onto the next element in in_ten (to stop the while loop at the end of in_ten)
        
        d = d+1
    
    return phot_rows

In [5]:
def mag_lookup(source_rows):
    """
    Takes the row of a source in the CMD count, then uses it to look up the related RA, Dec, & magnitudes
    in the phot_data table.  source_rows should come from the output of the corr_rows function.
    
    It is very similar to xy_lookup, but accesses all of the data associated with that ID instead of 
    just the desired x and y axes.
    
    This function is called "coord_lookup" in the 13July2018 TieredCat notebook.
    
    Call example:
        ID, RA, Dec, k36mag, k45mag, k58mag, k80mag, k24mag, Jmag, Hmag, Kmag, 
            jMINUSh, hMINUSk, jMINUSk = mag_lookup(source_rows)
    """
    
    ID = []
    RA = []
    Dec = []
    k36mag = []
    k45mag = []
    k58mag = []
    k80mag = []
    k24mag = []
    Jmag = []
    Hmag = []
    Kmag = []
    jMINUSh = []
    hMINUSk = []
    jMINUSk = []
    
    k = 0 # row counter
    for i in source_rows:
        ID.append(phot_data.ID.values[i])
        RA.append(phot_data.RA.values[i])
        Dec.append(phot_data.Dec.values[i])
        k36mag.append(phot_data.k36mag.values[i])
        k45mag.append(phot_data.k45mag.values[i])
        k58mag.append(phot_data.k58mag.values[i])
        k80mag.append(phot_data.k80mag.values[i])
        k24mag.append(phot_data.k24mag.values[i])
        Jmag.append(phot_data.Jmag.values[i])
        Hmag.append(phot_data.Hmag.values[i])
        Kmag.append(phot_data.Kmag.values[i])
        jMINUSh.append(phot_data.jMINUSh.values[i])
        hMINUSk.append(phot_data.hMINUSk.values[i])
        jMINUSk.append(phot_data.jMINUSk.values[i])
    
    return ID, RA, Dec, k36mag, k45mag, k58mag, k80mag, k24mag, Jmag, Hmag, Kmag, jMINUSh, hMINUSk, jMINUSk

In [6]:
def error_lookup(source_rows):
    """
    Takes the row of a source in the CMD count, then uses it to look up the related RA, Dec, & magnitudes
    in the phot_data table.  source_rows should come from the output of the corr_rows function.
    
    It is very similar to mag_lookup, but accesses all of the errors associated with the magnitudes instead of 
    just the magnitudes.  Use in concert with mag_lookup to produce a full set of lists to put into a catalog.
    
    
    Call example:
        e36mag, e45mag, e58mag, e80mag, e24mag, eJmag, eHmag, eKmag, = error_lookup(source_rows)
    """
    
    e36mag = []
    e45mag = []
    e58mag = []
    e80mag = []
    e24mag = []
    eJmag = []
    eHmag = []
    eKmag = []
    
    k = 0 # row counter
    for i in source_rows:
        e36mag.append(phot_data.e36mag.values[i])
        e45mag.append(phot_data.e45mag.values[i])
        e58mag.append(phot_data.e58mag.values[i])
        e80mag.append(phot_data.e80mag.values[i])
        e24mag.append(phot_data.e24mag.values[i])
        eJmag.append(phot_data.eJmag.values[i])
        eHmag.append(phot_data.eHmag.values[i])
        eKmag.append(phot_data.eKmag.values[i])
    
    return e36mag, e45mag,e58mag, e80mag, e24mag, eJmag, eHmag, eKmag

In [7]:
def create_colors(filename, colors):
    """
    Create the colors you want in a catalog.
    
    I only included the colors for the eight CMDs that we're currently using, but any color can be coded in.
    
    Call example:
        filename = '/Users/lgray/Documents/Phot_data/Red_Cand_Catalogs/24July2018/24July2018_LG_RedCand_8.csv'
        colors = ['jMINUSthreesix', 'threesixMINUSeightzero', 'fourfiveMINUSeightzero', 
                'hMINUSthreesix', 'hMINUSfourfive']
    """
    header = []
    
    for name in colors:
        catalog = pd.read_csv(filename)
        
        if name is 'jMINUSthreesix':
            catalog[name] = catalog.Jmag.values - catalog.k36mag.values
            catalog.to_csv(filename, index=False)
        if name is 'threesixMINUSeightzero':
            catalog[name] = catalog.k36mag.values - catalog.k80mag.values
            catalog.to_csv(filename, index=False)
        if name is 'fourfiveMINUSeightzero':
            catalog[name] = catalog.k45mag.values - catalog.k80mag.values
            catalog.to_csv(filename, index=False)
        if name is 'hMINUSthreesix':
            catalog[name] = catalog.Hmag.values - catalog.k36mag.values
            catalog.to_csv(filename, index=False)
        if name is 'hMINUSfourfive':
            catalog[name] = catalog.Hmag.values - catalog.k45mag.values
            catalog.to_csv(filename, index=False)

In [8]:
group = [in_eight, in_seven, in_six, in_five, in_four]
length = [230, 137, 215, 243, 250]

source_rows = corr_rows(group, length)

ID, RA, Dec, k36mag, k45mag, k58mag, k80mag, k24mag, Jmag, Hmag, Kmag, jMINUSh, hMINUSk, jMINUSk = mag_lookup(source_rows)
e36mag, e45mag,e58mag, e80mag, e24mag, eJmag, eHmag, eKmag = error_lookup(source_rows)
print(len(ID))

1075


In [9]:
# save data

filename = '/Users/lgray/Documents/Phot_data/Red_Cand_Catalogs/24July2018/RedCandTiers_24July2018_lauringray.csv'

f = open(filename, 'w')
writer = csv.writer(f)
#add heading
points_w_header = ['ID'] + ID

for val in points_w_header:
    writer.writerow([val])

f.close()

# list of other columns
cols = [RA, Dec, k36mag, e36mag, k45mag, e45mag, k58mag, e58mag, k80mag, e80mag, k24mag, e24mag, 
        Jmag, eJmag, Hmag, eHmag, Kmag, eKmag, jMINUSh, hMINUSk, jMINUSk]
headers = ['RA', 'Dec', 'k36mag', 'e36mag', 'k45mag', 'e45mag', 'k58mag', 'e58mag', 'k80mag', 'e80mag', 
           'k24mag', 'e24mag', 'Jmag','eJmag', 'Hmag', 'eHmag', 'Kmag', 'eKmag', 'jMINUSh', 'hMINUSk', 'jMINUSk']

c=0
for i in cols:
    data = pd.read_csv(filename)
    new_col = pd.DataFrame({headers[c]:i})
    c = c+1

    data= pd.concat([data, new_col], axis=1)
    data.to_csv(filename, index=False)

In [10]:
# add colors

colors = ['jMINUSthreesix', 'threesixMINUSeightzero', 'fourfiveMINUSeightzero', 'hMINUSthreesix', 'hMINUSfourfive']

create_colors(filename, colors)

### Save in separate catalogs

In [11]:
group = [in_eight]
length = [230]

source_rows = corr_rows(group, length)

ID, RA, Dec, k36mag, k45mag, k58mag, k80mag, k24mag, Jmag, Hmag, Kmag, jMINUSh, hMINUSk, jMINUSk = mag_lookup(source_rows)
e36mag, e45mag,e58mag, e80mag, e24mag, eJmag, eHmag, eKmag = error_lookup(source_rows)

In [12]:
filename = '/Users/lgray/Documents/Phot_data/Red_Cand_Catalogs/24July2018/24July2018_LG_RedCand_8.csv'

f = open(filename, 'w')
writer = csv.writer(f)
#add heading
points_w_header = ['ID'] + ID

for val in points_w_header:
    writer.writerow([val])

f.close()

# list of other columns
cols = [RA, Dec, k36mag, e36mag, k45mag, e45mag, k58mag, e58mag, k80mag, e80mag, k24mag, e24mag, 
        Jmag, eJmag, Hmag, eHmag, Kmag, eKmag, jMINUSh, hMINUSk, jMINUSk]
headers = ['RA', 'Dec', 'k36mag', 'e36mag', 'k45mag', 'e45mag', 'k58mag', 'e58mag', 'k80mag', 'e80mag', 
           'k24mag', 'e24mag', 'Jmag','eJmag', 'Hmag', 'eHmag', 'Kmag', 'eKmag', 'jMINUSh', 'hMINUSk', 'jMINUSk']

c=0
for i in cols:
    data = pd.read_csv(filename)
    new_col = pd.DataFrame({headers[c]:i})
    c = c+1

    data= pd.concat([data, new_col], axis=1)
    data.to_csv(filename, index=False)

In [13]:
# add colors

colors = ['jMINUSthreesix', 'threesixMINUSeightzero', 'fourfiveMINUSeightzero', 'hMINUSthreesix', 'hMINUSfourfive']

create_colors(filename, colors)

In [14]:
group = [in_seven]
length = [137]

source_rows = corr_rows(group, length)

ID, RA, Dec, k36mag, k45mag, k58mag, k80mag, k24mag, Jmag, Hmag, Kmag, jMINUSh, hMINUSk, jMINUSk = mag_lookup(source_rows)
e36mag, e45mag,e58mag, e80mag, e24mag, eJmag, eHmag, eKmag = error_lookup(source_rows)

In [15]:
filename = '/Users/lgray/Documents/Phot_data/Red_Cand_Catalogs/24July2018/24July2018_LG_RedCand_7.csv'

f = open(filename, 'w')
writer = csv.writer(f)
#add heading
points_w_header = ['ID'] + ID

for val in points_w_header:
    writer.writerow([val])

f.close()

# list of other columns
cols = [RA, Dec, k36mag, e36mag, k45mag, e45mag, k58mag, e58mag, k80mag, e80mag, k24mag, e24mag, 
        Jmag, eJmag, Hmag, eHmag, Kmag, eKmag, jMINUSh, hMINUSk, jMINUSk]
headers = ['RA', 'Dec', 'k36mag', 'e36mag', 'k45mag', 'e45mag', 'k58mag', 'e58mag', 'k80mag', 'e80mag', 
           'k24mag', 'e24mag', 'Jmag','eJmag', 'Hmag', 'eHmag', 'Kmag', 'eKmag', 'jMINUSh', 'hMINUSk', 'jMINUSk']

c=0
for i in cols:
    data = pd.read_csv(filename)
    new_col = pd.DataFrame({headers[c]:i})
    c = c+1

    data= pd.concat([data, new_col], axis=1)
    data.to_csv(filename, index=False)

In [16]:
# add colors

colors = ['jMINUSthreesix', 'threesixMINUSeightzero', 'fourfiveMINUSeightzero', 'hMINUSthreesix', 'hMINUSfourfive']

create_colors(filename, colors)

In [17]:
group = [in_six]
length = [215]

source_rows = corr_rows(group, length)

ID, RA, Dec, k36mag, k45mag, k58mag, k80mag, k24mag, Jmag, Hmag, Kmag, jMINUSh, hMINUSk, jMINUSk = mag_lookup(source_rows)
e36mag, e45mag,e58mag, e80mag, e24mag, eJmag, eHmag, eKmag = error_lookup(source_rows)

In [18]:
filename = '/Users/lgray/Documents/Phot_data/Red_Cand_Catalogs/24July2018/24July2018_LG_RedCand_6.csv'

f = open(filename, 'w')
writer = csv.writer(f)
#add heading
points_w_header = ['ID'] + ID

for val in points_w_header:
    writer.writerow([val])

f.close()

# list of other columns
cols = [RA, Dec, k36mag, e36mag, k45mag, e45mag, k58mag, e58mag, k80mag, e80mag, k24mag, e24mag, 
        Jmag, eJmag, Hmag, eHmag, Kmag, eKmag, jMINUSh, hMINUSk, jMINUSk]
headers = ['RA', 'Dec', 'k36mag', 'e36mag', 'k45mag', 'e45mag', 'k58mag', 'e58mag', 'k80mag', 'e80mag', 
           'k24mag', 'e24mag', 'Jmag','eJmag', 'Hmag', 'eHmag', 'Kmag', 'eKmag', 'jMINUSh', 'hMINUSk', 'jMINUSk']

c=0
for i in cols:
    data = pd.read_csv(filename)
    new_col = pd.DataFrame({headers[c]:i})
    c = c+1

    data= pd.concat([data, new_col], axis=1)
    data.to_csv(filename, index=False)

In [19]:
# add colors

colors = ['jMINUSthreesix', 'threesixMINUSeightzero', 'fourfiveMINUSeightzero', 'hMINUSthreesix', 'hMINUSfourfive']

create_colors(filename, colors)

In [20]:
group = [in_five]
length = [243]

source_rows = corr_rows(group, length)

ID, RA, Dec, k36mag, k45mag, k58mag, k80mag, k24mag, Jmag, Hmag, Kmag, jMINUSh, hMINUSk, jMINUSk = mag_lookup(source_rows)
e36mag, e45mag,e58mag, e80mag, e24mag, eJmag, eHmag, eKmag = error_lookup(source_rows)

In [21]:
filename = '/Users/lgray/Documents/Phot_data/Red_Cand_Catalogs/24July2018/24July2018_LG_RedCand_5.csv'

f = open(filename, 'w')
writer = csv.writer(f)
#add heading
points_w_header = ['ID'] + ID

for val in points_w_header:
    writer.writerow([val])

f.close()

# list of other columns
cols = [RA, Dec, k36mag, e36mag, k45mag, e45mag, k58mag, e58mag, k80mag, e80mag, k24mag, e24mag, 
        Jmag, eJmag, Hmag, eHmag, Kmag, eKmag, jMINUSh, hMINUSk, jMINUSk]
headers = ['RA', 'Dec', 'k36mag', 'e36mag', 'k45mag', 'e45mag', 'k58mag', 'e58mag', 'k80mag', 'e80mag', 
           'k24mag', 'e24mag', 'Jmag','eJmag', 'Hmag', 'eHmag', 'Kmag', 'eKmag', 'jMINUSh', 'hMINUSk', 'jMINUSk']

c=0
for i in cols:
    data = pd.read_csv(filename)
    new_col = pd.DataFrame({headers[c]:i})
    c = c+1

    data= pd.concat([data, new_col], axis=1)
    data.to_csv(filename, index=False)

In [22]:
# add colors

colors = ['jMINUSthreesix', 'threesixMINUSeightzero', 'fourfiveMINUSeightzero', 'hMINUSthreesix', 'hMINUSfourfive']

create_colors(filename, colors)

In [23]:
group = [in_four]
length = [250]

source_rows = corr_rows(group, length)

ID, RA, Dec, k36mag, k45mag, k58mag, k80mag, k24mag, Jmag, Hmag, Kmag, jMINUSh, hMINUSk, jMINUSk = mag_lookup(source_rows)
e36mag, e45mag,e58mag, e80mag, e24mag, eJmag, eHmag, eKmag = error_lookup(source_rows)

In [24]:
filename = '/Users/lgray/Documents/Phot_data/Red_Cand_Catalogs/24July2018/24July2018_LG_RedCand_4.csv'

f = open(filename, 'w')
writer = csv.writer(f)
#add heading
points_w_header = ['ID'] + ID

for val in points_w_header:
    writer.writerow([val])

f.close()

# list of other columns
cols = [RA, Dec, k36mag, e36mag, k45mag, e45mag, k58mag, e58mag, k80mag, e80mag, k24mag, e24mag, 
        Jmag, eJmag, Hmag, eHmag, Kmag, eKmag, jMINUSh, hMINUSk, jMINUSk]
headers = ['RA', 'Dec', 'k36mag', 'e36mag', 'k45mag', 'e45mag', 'k58mag', 'e58mag', 'k80mag', 'e80mag', 
           'k24mag', 'e24mag', 'Jmag','eJmag', 'Hmag', 'eHmag', 'Kmag', 'eKmag', 'jMINUSh', 'hMINUSk', 'jMINUSk']

c=0
for i in cols:
    data = pd.read_csv(filename)
    new_col = pd.DataFrame({headers[c]:i})
    c = c+1

    data= pd.concat([data, new_col], axis=1)
    data.to_csv(filename, index=False)

In [25]:
# add colors

colors = ['jMINUSthreesix', 'threesixMINUSeightzero', 'fourfiveMINUSeightzero', 'hMINUSthreesix', 'hMINUSfourfive']

create_colors(filename, colors)